# Retrieval augmented generation (RAG)

In [ ]:
# Download a file
import wget

file_path = "my_document.pdf"
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

In [ ]:
wget.download(doc_url, out=file_path)

In [ ]:
# Request configuration
import requests
import os

base_url = os.getenv("API_URL") # e.g. "http://localhost:8000/v1/"
api_key = os.getenv("API_KEY") # e.g. "your api key"

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

In [ ]:
# Upload a file
collection = "leo"
model = "BAAI/bge-m3"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

In [ ]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")

response.json()

In [ ]:
file_id = response.json()["data"][0]["id"]

In [ ]:
# Get tools
response = session.get(f"{base_url}/tools")

response.json()

In [ ]:
# Display tools parameters
for tool in response.json()["data"]:
    if tool["id"] == "BaseRag":
        print(tool["description"])

In [ ]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

In [ ]:
# Chat completions
user = "leo"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Qui est Ulrich Tan ?"}],
    "stream": False,
    "n": 1,
    "user": user,
    "tools": [
        {
            "function": {
                "name": "BaseRAG",
                "parameters": {
                    "embeddings_model": model,
                    "collections": [collection],
                    "k": 2,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)